SAMTOOLS MPILEUP ON MULTIPLE BAMS
=================================

In [1]:
! samtools --version

samtools 1.3.1
Using htslib 1.3.1
Copyright (C) 2016 Genome Research Ltd.


See ``mpileup`` [manual](http://samtools.sourceforge.net/mpileup.shtml).
``samtools mpileup`` requires:
- an index reference fasta
- indexed bam files 

The input bam files used in this example are assembled on hg19. 

We want to create a commands file for parallelization. We are using chromosome 22 as an example here. 
Parameters similar to GATK's default read filters were applied:
- depth 1000000 
- mapping quality 20 ([GATK haplotype caller](https://software.broadinstitute.org/gatk/gatkdocs/org_broadinstitute_gatk_tools_walkers_haplotypecaller_HaplotypeCaller.php) default)
- base quality 10 ([GATK haplotype caller](https://software.broadinstitute.org/gatk/gatkdocs/org_broadinstitute_gatk_tools_walkers_haplotypecaller_HaplotypeCaller.php) default)
- exclude read unmapped, not primary alignment, read fails platform/vendor quality checks, read is PCR or optical duplicate ([GATK haplotype caller](https://software.broadinstitute.org/gatk/gatkdocs/org_broadinstitute_gatk_tools_walkers_haplotypecaller_HaplotypeCaller.php) default)




In [3]:
%%bash
REFERENCE="/hackathon/Hackathon_Project_4/REFERENCE_GENOME/hg19.fa"
INDEXED_BAMPATHS="/hackathon/Hackathon_Project_4/ENCODE_DATA_GM12878/COMPLETED/rg_bams/*bam"
CHROMOSOME="chr22" # change this to any chromosome you like
OUTPUT_DIRECTORY="/hackathon/Hackathon_Project_4/MPILEUP/rg_bams"
MPILEUP_COMMANDS_FILE=${OUTPUT_DIRECTORY}/"mpileup_commands"


for bam in $(ls $INDEXED_BAMPATHS);
do
    # change the regex extension replacement 
    MPILEUP_COMMAND="samtools mpileup -d 1000000 --ff 1796 -q20  -Q10 -f $REFERENCE $bam -r $CHROMOSOME > ${OUTPUT_DIRECTORY}/$(basename $bam|sed 's/.bam/.mpileup/g') 2> ${OUTPUT_DIRECTORY}/$(basename $bam|sed 's/.bam/.log/g')"
    echo $MPILEUP_COMMAND
done > $MPILEUP_COMMANDS_FILE

wc -l ${MPILEUP_COMMANDS_FILE}
head -n 5 ${MPILEUP_COMMANDS_FILE}

479 /hackathon/Hackathon_Project_4/MPILEUP/rg_bams/mpileup_commands
samtools mpileup -d 1000000 --ff 1796 -q20 -Q10 -f /hackathon/Hackathon_Project_4/REFERENCE_GENOME/hg19.fa /hackathon/Hackathon_Project_4/ENCODE_DATA_GM12878/COMPLETED/rg_bams/ENCFF000ATH_chr22_ChIP-seq_rg.bam -r chr22 > /hackathon/Hackathon_Project_4/MPILEUP/rg_bams/ENCFF000ATH_chr22_ChIP-seq_rg.mpileup 2> /hackathon/Hackathon_Project_4/MPILEUP/rg_bams/ENCFF000ATH_chr22_ChIP-seq_rg.log
samtools mpileup -d 1000000 --ff 1796 -q20 -Q10 -f /hackathon/Hackathon_Project_4/REFERENCE_GENOME/hg19.fa /hackathon/Hackathon_Project_4/ENCODE_DATA_GM12878/COMPLETED/rg_bams/ENCFF000ATQ_chr22_ChIP-seq_rg.bam -r chr22 > /hackathon/Hackathon_Project_4/MPILEUP/rg_bams/ENCFF000ATQ_chr22_ChIP-seq_rg.mpileup 2> /hackathon/Hackathon_Project_4/MPILEUP/rg_bams/ENCFF000ATQ_chr22_ChIP-seq_rg.log
samtools mpileup -d 1000000 --ff 1796 -q20 -Q10 -f /hackathon/Hackathon_Project_4/REFERENCE_GENOME/hg19.fa /hackathon/Hackathon_Project_4/ENCODE_DATA_GM

Now we have a commands file we can use with ``parallel``. Because this is already generated, I'm commenting out the line that executes this. Uncomment to re-run.

In [4]:
%%bash
nproc 
CORENUM=32 # change number of cores here
OUTPUT_DIRECTORY="/hackathon/Hackathon_Project_4/MPILEUP/rg_bams"
MPILEUP_COMMANDS_FILE=${OUTPUT_DIRECTORY}/"mpileup_commands"
echo ${MPILEUP_COMMANDS_FILE}
time cat ${MPILEUP_COMMANDS_FILE} |parallel --gnu -j $CORENUM

64
/hackathon/Hackathon_Project_4/MPILEUP/rg_bams/mpileup_commands



real	61m45.991s
user	825m38.383s
sys	15m6.690s


The mpileup files should be generated in the output directory It took about an hour to run almost 500 samples with 32 cores.

In [5]:
%%bash
OUTPUT_DIRECTORY="/hackathon/Hackathon_Project_4/MPILEUP/rg_bams"
ls ${OUTPUT_DIRECTORY}/*mpileup|wc -l

479


Sanity Checks
-------------
Spot check for a few high confidence variants...
The two bams for initial testing, ENCFF000ARG, ENCFF000ARI are not included in the 479 so I couldn't reproduce the following with RG bams...

In [6]:
%%bash
OUTPUT_DIRECTORY="/hackathon/Hackathon_Project_4/MPILEUP" #
BENCHMARK_VCF="/hackathon/Hackathon_Project_4/BENCHMARK/Benchmark-Test-V1.vcf"

# using only column 2 because we predefined a specific chromosome; 
# might want to use CHROM and POS columns from the VCFs otherwise
grep -f <(cut -f 2 ${BENCHMARK_VCF}) ${OUTPUT_DIRECTORY}/ENCFF000ARG.mpileup > ${OUTPUT_DIRECTORY}/ENCFF000ARG.benchmarked.mpileup
grep -f <(cut -f 2 ${BENCHMARK_VCF}) ${OUTPUT_DIRECTORY}/ENCFF000ARI.mpileup > ${OUTPUT_DIRECTORY}/ENCFF000ARI.benchmarked.mpileup
wait
wc -l ${OUTPUT_DIRECTORY}/ENCFF000ARG.benchmarked.mpileup
wc -l ${OUTPUT_DIRECTORY}/ENCFF000ARI.benchmarked.mpileup
wc -l ${BENCHMARK_VCF}

0 /hackathon/Hackathon_Project_4/MPILEUP/rg_bams/ENCFF000ARG.benchmarked.mpileup
0 /hackathon/Hackathon_Project_4/MPILEUP/rg_bams/ENCFF000ARI.benchmarked.mpileup
25 /hackathon/Hackathon_Project_4/BENCHMARK/Benchmark-Test-V1.vcf


grep: /hackathon/Hackathon_Project_4/MPILEUP/rg_bams/ENCFF000ARG.mpileup: No such file or directory
grep: /hackathon/Hackathon_Project_4/MPILEUP/rg_bams/ENCFF000ARI.mpileup: No such file or directory


In [6]:
%%bash
OUTPUT_DIRECTORY="/hackathon/Hackathon_Project_4/MPILEUP"
head ${OUTPUT_DIRECTORY}/ENCFF000ARG.benchmarked.mpileup
echo
head ${OUTPUT_DIRECTORY}/ENCFF000ARI.benchmarked.mpileup


chr22	18268130	C	13	.T....T..TT..	4<<<79<<<<<<<
chr22	19132325	A	14	...G...CG.GGG.	69<<70;1<2<<.<
chr22	24585835	T	7	,c,cccc	<</<<..
chr22	26727129	C	13	TT.T.,...T.T^0.	<<<<<9<<<<<<;
chr22	30787190	t	11	.CC.......C	:<<55<<<<<<
chr22	33396853	C	11	..TT.TT.,TT	260:<<<7<<<
chr22	33396904	C	2	t,	<8
chr22	33397045	T	9	,$cccccc,c	<<<<<<<<<
chr22	33414359	G	11	AAAAAAA,A.A	9994<<7<<<<
chr22	33445237	A	12	.C.C...CCC.C	9<<<;;<<<7<<

chr22	18268130	C	19	T...t..T.TT.....T.^g.	4:=8B9B<7<=<B4A<<;<
chr22	19132325	A	18	.....G...GGGG.GG.^b.	8=B@A<;<B8CAC5BCB>
chr22	24585835	T	16	,CCCC,,c,,,,,,cC	<17<<BA<><AB;B;<
chr22	26727129	C	21	.T.,TT.T....,,t...TTT	A@=?8364<<<6B1CA@A8<B
chr22	30787190	t	14	CC.CcC.CC...CC	8B8BBB5AB<A<01
chr22	33396853	C	13	T..Tt.t.TTTT.	82<5<<<<<=>5<
chr22	33396904	C	15	tt,,.,,,.,t,,,^f,	<A<<89<9B<<<<14
chr22	33397045	T	7	cc,,c,,	<BB<<B7
chr22	33414359	G	12	,A.A...A.,,A	<;<<<<B<<@<<
chr22	33445237	A	19	CCC.CCC..cC..CC...,	<.@3<=;A1C@;<5CCCC7


SAMTOOLS MPILEUP WITH A SINGLE MERGED BAM
=========================================

This merged bam ``/hackathon/Hackathon_Project_4/VariantCall_HAPLOTYPE/merged_rg.chr22.bam`` is constructed with ENCFF000ARG and ENCFF000ARI. The read groups are tagged so that we know the source.

In [7]:
%%bash

REFERENCE="/hackathon/Hackathon_Project_4/REFERENCE_GENOME/hg19.fa"
bam="/hackathon/Hackathon_Project_4/VariantCall_HAPLOTYPE/merged_rg.chr22.bam" # TODO: change this to bam when everything is indexed
CHROMOSOME="chr22" # change this to any chromosome you like
OUTPUT_DIRECTORY="/hackathon/Hackathon_Project_4/MPILEUP/merged"

samtools mpileup -f $REFERENCE $(echo $bam|sed 's/.bai//g') -r $CHROMOSOME > ${OUTPUT_DIRECTORY}/$(basename $bam|sed 's/.bam/.mpileup/g')
   

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


Next we pull out the subset of positions from the benchmark data set.

In [8]:
%%bash
OUTPUT_DIRECTORY="/hackathon/Hackathon_Project_4/MPILEUP/merged"
BENCHMARK_VCF="/hackathon/Hackathon_Project_4/BENCHMARK/Benchmark-Test-V1.vcf"

grep -f <(cut -f 2 ${BENCHMARK_VCF}) ${OUTPUT_DIRECTORY}/merged_rg.chr22.mpileup > ${OUTPUT_DIRECTORY}/merged_rg.chr22.benchmarked.mpileup


COMPARING COVERAGE BETWEEN THE TWO APPROACHES
=============================================

In [9]:
import os
import pandas

out_dir = "/hackathon/Hackathon_Project_4/MPILEUP"
ENCFF000ARG = os.path.join(out_dir, "ENCFF000ARG.benchmarked.mpileup")
ENCFF000ARI = os.path.join(out_dir, "ENCFF000ARI.benchmarked.mpileup")
merged = os.path.join(out_dir, "merged", "merged_rg.chr22.benchmarked.mpileup")

ENCFF000ARG_data = pandas.read_table(ENCFF000ARG, header=None)
ENCFF000ARI_data = pandas.read_table(ENCFF000ARI, header=None)
merged_data = pandas.read_table(merged, header=None)
assert all(ENCFF000ARG_data[1]==ENCFF000ARI_data[1])
assert all(ENCFF000ARI_data[1]==merged_data[1])
pandas.DataFrame({"sum_of_cov": ENCFF000ARG_data[3]+ENCFF000ARI_data[3], "merged": merged_data[3]})



,merged,sum_of_cov
0,32,32
1,32,32
2,23,23
3,34,34
4,25,25
5,24,24
6,17,17
7,16,16
8,23,23
9,31,31


SUM OF COVERAGE
===============

In [11]:
import os
identifiers="/hackathon/Hackathon_Project_4/ENCODE_DATA_GM12878/COMPLETED/filesToUse.txt"
output_file="/hackathon/Hackathon_Project_4/MPILEUP/samples.txt"
with open(identifiers, 'r') as ifile, open(output_file, 'w') as ofile:
    for row in ifile:
        orow = os.path.join("/hackathon/Hackathon_Project_4/MPILEUP", row.strip() + ".mpileup" ) + "\n"
        ofile.write(orow)

In [4]:
# script to generate a file with CHROM, POS-1, POS, sum of coverage
input_file="/hackathon/Hackathon_Project_4/DEPTH/depths50chr22.txt"
output_file="/hackathon/Hackathon_Project_4/DEPTH/cov.txt"

import csv

with open(input_file, 'r') as ifile, open(output_file, 'w') as ofile:
    csvreader = csv.reader(ifile, delimiter="\t")
    csvwriter = csv.writer(ofile, delimiter="\t")
    counter = 0
    for line in csvreader:
        cov_sum = sum(map(int,line[2:]))
        csvwriter.writerow([ str(x) for x in [line[0], int(line[1])-1, line[1], cov_sum]])
        counter += 1
        if counter%100000==0:
            print("Parsed {} lines...".format(counter))

Parse 100000 lines...
Parse 200000 lines...
Parse 300000 lines...
Parse 400000 lines...
Parse 500000 lines...
Parse 600000 lines...
Parse 700000 lines...
Parse 800000 lines...
Parse 900000 lines...
Parse 1000000 lines...
Parse 1100000 lines...
Parse 1200000 lines...
Parse 1300000 lines...
Parse 1400000 lines...
Parse 1500000 lines...
Parse 1600000 lines...
Parse 1700000 lines...
Parse 1800000 lines...
Parse 1900000 lines...
Parse 2000000 lines...
Parse 2100000 lines...
Parse 2200000 lines...
Parse 2300000 lines...
Parse 2400000 lines...
Parse 2500000 lines...
Parse 2600000 lines...
Parse 2700000 lines...
Parse 2800000 lines...
Parse 2900000 lines...
Parse 3000000 lines...
Parse 3100000 lines...
Parse 3200000 lines...
Parse 3300000 lines...
Parse 3400000 lines...
Parse 3500000 lines...
Parse 3600000 lines...
Parse 3700000 lines...
Parse 3800000 lines...
Parse 3900000 lines...
Parse 4000000 lines...
Parse 4100000 lines...
Parse 4200000 lines...
Parse 4300000 lines...
Parse 4400000 lines.